In [1]:
!pip install py7zr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.9/412.9 kB 25.9 MB/s eta 0:00:00


In [2]:
import py7zr

# Путь к архиву
archive_path = '/kaggle/input/cifar-10/train.7z'

# Папка для распаковки
extract_to = '/kaggle/working/'

# Распаковка архива
with py7zr.SevenZipFile(archive_path, mode='r') as z:
    z.extractall(path=extract_to)

print(f"Архив распакован в {extract_to}")


Архив распакован в /kaggle/working/


In [3]:
import py7zr

# Путь к архиву
archive_path = '/kaggle/input/cifar-10/test.7z'

# Папка для распаковки
extract_to = '/kaggle/working/'

# Распаковка архива
with py7zr.SevenZipFile(archive_path, mode='r') as z:
    z.extractall(path=extract_to)

print(f"Архив распакован в {extract_to}")

Архив распакован в /kaggle/working/


In [4]:
import pandas as pd
import numpy as np
labels = pd.read_csv('/kaggle/input/cifar-10/trainLabels.csv')
labels

,id,label
0,1,frog
1,2,truck
2,3,truck
3,4,deer
4,5,automobile
...,...,...
49995,49996,bird
49996,49997,frog
49997,49998,truck
49998,49999,automobile


In [5]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

le.fit(labels['label'])

labels['label'] = le.transform(labels['label'])

In [6]:
import os
from torch.utils.data import Dataset
from PIL import Image

In [7]:
d={}

for i in range(len(labels)):
    d[str(labels.loc[i,'id'])] = labels.loc[i,'label']

In [8]:
class CustomImageDataset(Dataset):
    def __init__(self,image_dir,labels,transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_paths = [os.path.join(image_dir,fname) for fname in os.listdir(image_dir) if fname.endswith('.png')]
    def __len__(self):
        return len(self.image_paths)
    def __getitem__(self,idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')

        filename = os.path.basename(image_path)
        label = d[filename[0:len(filename)-4]]

        if self.transform:
            image = self.transform(image)

        return image,label


In [9]:
from torch.utils.data import DataLoader
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize(((224,224))),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = CustomImageDataset(image_dir='/kaggle/working/train',labels=labels,transform = transform)
dataloader = DataLoader(dataset,batch_size=8,shuffle=True)



In [10]:
import torch.nn as nn
import torch.optim as optim

In [11]:
data_for_train = dataloader

In [12]:
import torch.nn.functional as F
import torch

In [13]:
from torchvision import models, transforms

In [14]:
import torch.nn as nn
import torch.nn.functional as F


# Модель
class ResNetClassifier(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.base = models.resnet18(pretrained=True)

        # Разморозим только последние слои
        for name, param in self.base.named_parameters():
            if 'layer4' in name or 'fc' in name:
                param.requires_grad = True
            else:
                param.requires_grad = False

        self.base.fc = nn.Linear(self.base.fc.in_features, num_classes)

    def forward(self, x):
        return self.base(x)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNetClassifier(num_classes=10).to(device)

# Loss + Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

# Обучение
epochs = 2
for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for images, labels in data_for_train:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(data_for_train):.4f}, Accuracy: {correct/total:.4f}")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 171MB/s]


Epoch 1/2, Loss: 0.5293, Accuracy: 0.8217
Epoch 2/2, Loss: 0.2805, Accuracy: 0.9050


In [15]:
sample = pd.read_csv('/kaggle/input/cifar-10/sampleSubmission.csv')

In [16]:
class ComfortableData(Dataset):
    def __init__(self,image_dir,transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.labels = labels
        self.image_paths = [os.path.join(image_dir,fname) for fname in os.listdir(image_dir) if fname.endswith('.png')]
    def __len__(self):
        return len(self.image_paths)
    def __getitem__(self,idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path)

        filename = os.path.basename(image_path)
        
        if self.transform:
            image = self.transform(image)

        return image, os.path.basename(image_path)[:-4]


In [17]:
comfort_data = ComfortableData(image_dir='/kaggle/working/test',transform = transform)
data_for_test = DataLoader(comfort_data, batch_size = 8, shuffle=False)
    
model.eval()  # Важный шаг
preds = [0]*(len(sample)+1)

with torch.no_grad():
    for image, ID in data_for_test:
        image = image.to(device)
        predictions = model(image)  # (batch_size, 10)
        predicted_classes = torch.argmax(predictions, dim=1)  # (batch_size,)
        
        # Сохраняем предсказания по ID
        for i in range(len(ID)):
            preds[int(ID[i])] = le.inverse_transform([predicted_classes[i].item()])[0]
            
    

In [18]:
import pandas as pd

# Преобразуем в DataFrame
df = pd.DataFrame({
    'id': sample['id'],
    'label': preds[1:len(preds)]
})
df.to_csv("submission.csv", index=False)


In [19]:
for i in os.listdir('/kaggle/working/test'):
    print(i)
    break


188742.png
